# Mount Google Drive

In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = "cs231n-project"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


# GPU if available

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


# Load Dataset

In [3]:
dataset_path = f"/content/drive/My Drive/cs231n-project/dataset/Taskent"

dataset_size = 21
# test_indices = [1,2]
# train_indices = [i for i in range(dataset_size) if not i in test_indices]
# train_indices = [0,3,4]

In [4]:
from image_dataset import *

In [5]:
dataset = PetroTrainTestSplitDataset(folder_path=dataset_path)

train_dataset = dataset['train']
test_dataset = dataset['test']

__init__ PetroSubImageDataset: /content/drive/My Drive/cs231n-project/dataset/Taskent
         , image_indices=None
         , sub_image_size=480
__init__ BaseSubImageDataset with: /content/drive/My Drive/cs231n-project/dataset/Taskent/img
         , image_indices=None
         , sub_image_size=480
         , mask=False
__init__ BaseSubImageDataset with: /content/drive/My Drive/cs231n-project/dataset/Taskent/masks_machine
         , image_indices=None
         , sub_image_size=480
         , mask=True


/content/drive/My Drive/cs231n-project/image_dataset.py:211: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  images = torch.stack([torch.from_numpy(img) for img in images])


# Load DINO Model and helper functions

In [6]:
from torch.utils.data import DataLoader
from clip_dino import DINOSegmentation, compute_iou
from dino_model import DINOPatchClassifier
from tqdm import tqdm
from einops import rearrange
import numpy as np

In [7]:
# Load smallest dino model. ViT-S/8. Here ViT-S has ~22M parameters and
# works on 8x8 patches.
dino_model = torch.hub.load('facebookresearch/dino:main', 'dino_vits8')
dino_model.eval().to(device)

Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall8_pretrain.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 153MB/s]


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(8, 8), stride=(8, 8))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (head): Identity()
)

In [8]:
from torchvision import transforms as T

transform = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

# transform = T.Compose([
#     # T.Resize((480, 480)),
#     T.ToTensor(),
#     T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
# ])


In [9]:
def get_dino_tokens_batch(X_batch):

    # X_batch.shape = [batch_size,480,480,3]

    X_batch = X_batch.float() / 255.0 # normalize
    # X_batch = X_batch.permute(0,3,1,2) # [N,H,W,C] -> [N,C,H,W]
    X_transform = torch.stack([
        transform(x) for x in X_batch
    ])
    X_transform = X_transform.to(device)

    w, h = X_transform.shape[2:]
    # 480,480

    with torch.no_grad():
        attn = dino_model.get_last_selfattention(X_transform)[:, :, 0, 1:]
        # (N,6,3600)
        # print(f"attn.shape={attn.shape}")
        nh, tokens = attn.shape[1:]
        w_feat, h_feat = w // 8, h // 8
        attn = attn.reshape(-1, nh, w_feat, h_feat)
        attn = torch.nn.functional.interpolate(attn, scale_factor=8, mode="nearest").cpu().numpy()
        all_tokens = dino_model.get_intermediate_layers(X_transform, n=1)[0]  # (N, 1+pixels, D)
    return all_tokens.cpu()

In [10]:
# def get_patchwise_biotic_ornot(Y_batch):
#     """
#     Input:
#         Y_batch: (N, 480, 480) segmentation labels as a NumPy array or torch.Tensor

#     Output:
#         (N, 60, 60) torch.Tensor where each value is 1 (biotic) or 0 (non-biotic)
#     """

#     if isinstance(Y_batch, torch.Tensor):
#         Y_batch = Y_batch.cpu().numpy()

#     # Define class sets
#     abiotic = {1, 2, 11, 17}
#     biotic = {3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16}
#     scale_bar = {8}

#     biotic_ornot_mask = np.full_like(Y_batch, fill_value=3, dtype=np.uint8)
#     biotic_ornot_mask[np.isin(Y_batch, list(abiotic))] = 0
#     biotic_ornot_mask[np.isin(Y_batch, list(biotic))] = 1
#     biotic_ornot_mask[np.isin(Y_batch, list(scale_bar))] = 0

#     # Rearrange to 8x8 patches
#     patches = rearrange(biotic_ornot_mask, 'n (h ph) (w pw) -> n h w ph pw', ph=8, pw=8)
#     # patches shape = (N, 60, 60, 8, 8)

#     # Check if any pixel in each 8x8 patch is biotic (== 1)
#     patchwise_biotic = (patches == 1).any(axis=(-1, -2)).astype(np.uint8)
#     # shape = (N, 60, 60)

#     return torch.tensor(patchwise_biotic)

In [11]:
import numpy as np
import torch
from einops import rearrange
from scipy.stats import mode

def get_patchwise_mode(Y_batch):

    if isinstance(Y_batch, torch.Tensor):
        Y_batch = Y_batch.cpu().numpy()

    # Rearrange to 8x8 patches
    patches = rearrange(Y_batch, 'n (h ph) (w pw) -> n h w (ph pw)', ph=8, pw=8)
    # shape: (N, H//8, W//8, 64)

    # Compute mode along last axis
    patch_modes = mode(patches, axis=-1).mode  # shape: (N, H//8, W//8)

    return torch.tensor(patch_modes, dtype=torch.uint8)

## WandB

In [12]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: thisisrmak (thisisrmak-stanford) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Run test/train loop

In [ ]:
wandb.init(
    project="dinov2-biotic-nb-08",
    name="run-patchwise-mode",  # Optionally change per experiment
    config={
        "num_classes": 18,
        "batch_size": 32,
        "num_iters": 50,
        "hidden_dim": 768,
        "lr": 1e-3
    }
)

config = wandb.config

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size)

model = DINOPatchClassifier(num_classes=config.num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
loss_fn = torch.nn.CrossEntropyLoss()

for iter in range(config.num_iters):

    ### TRAINING ###

    model.train()
    total_train_loss = 0.0
    correct_train = 0
    total_train = 0
    total_train_iou = 0.0

    pbar = tqdm(train_dataloader, desc=f"Train Epoch {iter+1}/{config.num_iters}")

    for batch in pbar:
        X_batch = batch[:, :-1]
        Y_batch = batch[:, -1]

        Y = get_patchwise_mode(Y_batch)
        Y = rearrange(Y, 'n h w -> (n h w)')
        Y = Y.to(device)

        with torch.no_grad():
            X = get_dino_tokens_batch(X_batch)  # [N, 3601, D=384]
            X = X[:, 1:, :]  # drop CLS token
            X = rearrange(X, 'n p d -> (n p) d')
            X = X.to(device)

        optimizer.zero_grad()
        logits = model(X)
        loss = loss_fn(logits, Y)
        loss.backward()
        optimizer.step()

        # Stats
        batch_loss = loss.item()
        total_train_loss += batch_loss

        preds = torch.argmax(logits, dim=1)
        correct_train += (preds == Y).sum().item()
        total_train += Y.size(0)

        batch_iou = compute_iou(preds.cpu().numpy(), Y.cpu().numpy(), config.num_classes)
        total_train_iou += batch_iou

        pbar.set_postfix(loss=f"{batch_loss:.2f}", iou=f"{batch_iou:.2f}")

    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_train_accuracy = correct_train / total_train
    avg_train_iou = total_train_iou / len(train_dataloader)



    # print(f"Epoch {iter+1}: Train Loss={avg_train_loss:.4f}, Accuracy={avg_train_accuracy:.4f}, IoU={avg_train_iou:.4f}")

     ### VALIDATION ###

    model.eval()
    total_val_loss = 0.0
    correct_val = 0
    total_val = 0
    total_val_iou = 0.0

    with torch.no_grad():
        pbar = tqdm(test_dataloader, desc=f"Val Epoch {iter+1}/{config.num_iters}")

        for batch in pbar:
            X_batch = batch[:, :-1]
            Y_batch = batch[:, -1]

            Y = get_patchwise_mode(Y_batch)
            Y = rearrange(Y, 'n h w -> (n h w)').to(device)

            X = get_dino_tokens_batch(X_batch)
            X = X[:, 1:, :]
            X = rearrange(X, 'n p d -> (n p) d').to(device)

            logits = model(X)
            loss = loss_fn(logits, Y)
            batch_loss = loss.item()
            total_val_loss += batch_loss

            preds = torch.argmax(logits, dim=1)
            correct_val += (preds == Y).sum().item()
            total_val += Y.size(0)

            batch_iou = compute_iou(preds.cpu().numpy(), Y.cpu().numpy(), config.num_classes)
            total_val_iou += batch_iou

            pbar.set_postfix(loss=f"{batch_loss:.2f}", iou=f"{batch_iou:.2f}")

    avg_val_loss = total_val_loss / len(test_dataloader)
    avg_val_accuracy = correct_val / total_val
    avg_val_iou = total_val_iou / len(test_dataloader)

    # ---------- LOGGING ----------
    wandb.log({
        "epoch": iter + 1,
        "train/loss": avg_train_loss,
        "train/accuracy": avg_train_accuracy,
        "train/iou": avg_train_iou,
        "val/loss": avg_val_loss,
        "val/accuracy": avg_val_accuracy,
        "val/iou": avg_val_iou
    })
    print()

    print(f"Epoch {iter+1}: "
          f"Train Loss={avg_train_loss:.4f}, Acc={avg_train_accuracy:.4f}, IoU={avg_train_iou:.4f} | "
          f"Val Loss={avg_val_loss:.4f}, Acc={avg_val_accuracy:.4f}, IoU={avg_val_iou:.4f}")

Val Epoch 1/50: 100%|██████████| 8/8 [01:06<00:00,  8.27s/it, iou=0.04, loss=2.06]



Epoch 1: Train Loss=2.3537, Acc=0.4699, IoU=0.0320 | Val Loss=1.9126, Acc=0.4644, IoU=0.0333


Val Epoch 2/50: 100%|██████████| 8/8 [01:07<00:00,  8.44s/it, iou=0.05, loss=1.26]



Epoch 2: Train Loss=1.5335, Acc=0.4908, IoU=0.0332 | Val Loss=1.1716, Acc=0.5990, IoU=0.0457


Val Epoch 3/50: 100%|██████████| 8/8 [01:06<00:00,  8.33s/it, iou=0.09, loss=1.02]


...


Epoch 48: Train Loss=0.5010, Acc=0.8089, IoU=0.1105 | Val Loss=0.7648, Acc=0.7494, IoU=0.0948


Val Epoch 49/50: 100%|██████████| 8/8 [01:03<00:00,  7.95s/it, iou=0.16, loss=0.75]



Epoch 49: Train Loss=0.4823, Acc=0.8180, IoU=0.1112 | Val Loss=0.7410, Acc=0.7563, IoU=0.0967


Val Epoch 50/50: 100%|██████████| 8/8 [01:03<00:00,  7.95s/it, iou=0.17, loss=0.73]


Epoch 50: Train Loss=0.4669, Acc=0.8246, IoU=0.1131 | Val Loss=0.7365, Acc=0.7642, IoU=0.0971
